# Reading FITS images from LB telescopes
First tests with Astropy

In [115]:
import os, sys, glob
from pathlib import Path
import datetime as dt
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib qt

from astropy.io import fits
from astropy import units as u
from astropy.wcs import WCS
from astropy.wcs.wcsapi import SlicedLowLevelWCS
from astropy.wcs.utils import skycoord_to_pixel
from astropy.visualization.wcsaxes import add_scalebar
from astropy.visualization import SimpleNorm, simple_norm
from astropy.visualization import make_lupton_rgb
from astropy.coordinates import Angle
from astropy.coordinates import SkyCoord
from astropy.nddata import Cutout2D




In [ ]:
class image_viewer:
    def __init__(self, directory: str = '',
               list_available: bool = False,
               folder_list = False):
        """Class to quickly open FITS images. Searches in given directory.
        
        Attributes
        ---------
        directory : str
            Directory where images are stored. If none given look in current working directory.

        list_available : bool : False
            Wether to print the resulting dataframe of found images or not

        folder_list : optional, list of str
            Extra directories to inspect for images and save their folder path from working directory
        
        Methods
        --------
        return_index()
            Returns the image path and index in the datafile given one or the other.
        
        header_info()
            Method to view general header info.

        view()
            Method to view images.
    """
        self.folder_list = folder_list
        print('Current working directory: ' + os.getcwd())
        if directory=='':
            directory = os.getcwd()
        if directory != os.getcwd():
            self.dir_img = os.path.join(os.getcwd(),directory)
        else: self.dir_img = directory
        print('Image directory defined: ' + self.dir_img)

        # list of images
        files = list(Path(self.dir_img).glob('*.fits'))
        folder_found = ['']*len(files)
        if folder_list!= False:
            for fl in folder_list:
                fi = list(Path(os.path.join(self.dir_img, fl)).glob('*.fits'))
                files=files+fi
                folder_found =folder_found+[fl]*len(fi)

        files_data = []
        
        for k, f in enumerate(files):
            try:
                name = f.name
                path = str(f.resolve())
                try: telescope, camera, date_time, object, filter = name.split('_')
                except: print('ERROR WITH FILENAME FORMAT CONVENTION EXPECTED')
                size_MB = f.stat().st_size / 1e6
                created = pd.to_datetime(f.stat().st_ctime, unit="s")
                files_data.append({"filename": name, "path": path, "telescope": telescope, 'camera': camera,
                                   "object": object, "filter": filter[:-5], "size_MB": size_MB,
                                   "date_time": pd.to_datetime(date_time, format='%Y-%m-%d-%H-%M-%S-%f'),
                                   "folder_found": folder_found[k]})
            except: print('Error with file: %s'%f)
                
        if len(files)==0:
            print('WARNING: NO IMAGE FILES FOUND')
            return
        self.df_files = pd.DataFrame(files_data).sort_values("filename").reset_index(drop=True)

        # print available images if requested
        if list_available:
            print(self.df_files)
        print('Total number of images found: ', len(self.df_files))
    
    def return_index(self, image):
        """
        Returns the image path and index in the datafile given one or the other.

        Parameters
        ----------
        image: int / str
            int - image index in datafile \n
            str - image path
        """
        if type(image)==int:
            image_str = self.df_files.iloc[image].filename
            image_int = image
        else: 
            image_str = image
            try: image_int = self.df_files.index[self.df_files['filename']==image].to_list()[0]
            except:
                print('\n ERROR: FILENAME NOT FOUND')
                return
        if self.folder_list != False:
            # Extract date, but considering that images from the early am are saved in the folder
            # corresponding to the previous day
            # ------- Old techinque, not necessary maybe ------
            # time = self.df_files.iloc[image_int].date_time
            # if time.hour < 12:
            #     time += dt.timedelta(days=-1)
            # folder_name = str(time.date())
            # -----------------------------------------------
            folder_name = self.df_files.iloc[image_int].folder_found
            image_str = os.path.join(folder_name, image_str)
        return image_str, image_int


    def header_info(self, image,
                    interesting_keys = ['INSTRUME', 'OBJECT', 'FILTER', 'INTEGT', 'DATE-OBS',
                                        'RA', 'DEC', 'NAXIS1', 'NAXIS2', 'SCALE', 'FOVX', 'FOVY',
                                        'CCW', 'CRPIX1', 'CRPIX2']
                                        ):
        """Method to view general header info.
        
        Parameters
        ----------
        image : int / str
            int - index of desired file in dataframe \n
            string - path to desired fits file
            
        interesting_keys: list / 'all'
            list - list of strings with header keyword \n
            'all' - will print the whole header
        """
        image_str, image_int = self.return_index(image)
        
        # Extracting data from header
        with fits.open(os.path.join(self.dir_img, image_str)) as hdul:
            heads = hdul[0].header
            hdul.close()
        # printing basic header info
        print('Image: %s'%image_str)
        print('\n   --- HEADER DATA ---')
        try:
            if type(interesting_keys) == str and interesting_keys=='all':
                print(repr(heads))
            else:
                for k in interesting_keys:
                    if heads.comments[k]!='':
                        print(k, ' = ', heads[k], '  ---  ', heads.comments[k])
                    else:
                        print(k, ' = ', heads[k])
        except:
            print('WARNING: WRONG interesting_keys PARAMETER.')
            print('         Try a list with the strings of header keys or just the string \'all\'')


    def view(self, image,
             centered = True, 
             zoom = False,
             stretch = 'linear',
             percentile = None,
             vminmax = (None, None),
             cmap = 'gray',
             scalebar_arcsec = 5
             ):
        """Method to view images.

        Parameters
        ---------
        image : int / string
            int - index of desired file in dataframe \n
            string - path to desired fits file

        centered : True / tuple 
            (x,y) - int for pix coordinates \n
            (RA, DEC) - wcs coordinates. Accepting both strings or angle values

        zoom : False / Value / Tuple 
            int / (int, int) - pixel size in x and y axis \n
            Angle / (Angle, Angle) - angular size in RA and DEC
        
        stretch : str
            Image stretch to enhance detail visualization \n
            ``linear``, ``sqrt``, ``power``, ``log``, ``sinh``, ``asinh``
        
        percentile : int / tuple
            ``int`` - Middle percentile of values to consider for normalization; 
            ``tuple`` - Lower and upper percentile of values to consider for normalization
        
        vminmax : tuple
            Min and max pixel values for normalization. Overrides ``percentile``.
            If set as None, keeps the absolute min or max of image

        cmap : str
            Select the desired colormap for the image

        scalebar_arcsec : int
            Angular size of scalebar in arcsec units
        """
        image_str, image_int = self.return_index(image)
        print('Viewing ', image_str)

        # Extracting data from header
        with fits.open(os.path.join(self.dir_img, image_str)) as hdul:
            data = hdul[0].data.astype(np.float32)
            heads = hdul[0].header
            wcs = WCS(heads)
            hdul.close()
        
        # obtaining central px coordinates
        x_shape = data.shape[1]
        y_shape = data.shape[0]
        if centered == True:
            center_px = (x_shape//2, y_shape//2)
        if type(centered)==tuple:
            if type(centered[0]) == int: # input in px units
                center_px = tuple(centered)
            if type(centered[0]) == str: # input in str to be converted to deg
                center_angle = SkyCoord(centered[0], centered[1], frame = 'icrs')
                center_px = skycoord_to_pixel(center_angle, wcs, origin=0)
        
        # setting zoom
        if zoom == False:
            zoom = (x_shape, y_shape)
        if type(zoom) == str:
            zoom = Angle(zoom)
        if type(zoom)== tuple:
            if type(zoom[0]) == str:
                zoom = (Angle(zoom[0]), Angle(zoom[1]))
        if type(zoom)==tuple:
            zoom = zoom[::-1]
        
        # slicing image
        try:
            cutout = Cutout2D(data, position = center_px, size = zoom, wcs = wcs)
        except:
            print('\n --- \nERROR: the cutout region is outside of the image.')
            return

        # WCS projection
        fig, ax = plt.subplots(subplot_kw=dict(projection=cutout.wcs))

        # norm definition
        if type(percentile) == int or percentile == None:
            percentile_minmax = (None, None)
        if type(percentile) == tuple:
            percentile_minmax = percentile
            percentile = None
        if stretch not in {'linear', 'sqrt', 'power', 'log', 'sinh', 'asinh'}:
            print('ERROR: Stretch should be one of \'linear\', \'sqrt\', \'power\', \'log\', \'sinh\', \'asinh\'')
            plt.close()
            return
        norm = simple_norm(cutout.data, stretch = stretch, 
                           vmin = vminmax[0], vmax = vminmax[1],
                           percent = percentile,
                           min_percent = percentile_minmax[0],
                           max_percent = percentile_minmax[1])
        
        cax = ax.imshow(cutout.data,
                        norm = norm, origin = 'lower',
                        cmap = cmap)
        cbar = plt.colorbar(cax)
        cbar.set_label('ADU', rotation=270, labelpad=15)
        cbar.ax.tick_params(labelsize=10)

        # Scale bar choosing color depending on luminance of cmap
        scalebar_angle = scalebar_arcsec/3600*u.deg
        rgba = plt.get_cmap(cmap)(0.0)
        luminance = 0.299*rgba[0] + 0.587*rgba[1] + 0.114*rgba[2]
        scalebar_color = 'white' if luminance < 0.5 else 'black'
        add_scalebar(ax, scalebar_angle, label="%s arcsec"%str(scalebar_arcsec), color=scalebar_color)
        # Axis and title
        ax.set(xlabel='RA', ylabel='Dec')
        ax.coords.grid(color='gray', alpha=0.5, linestyle='solid')
        title_str = '$\\bf{Object}$: %s - $\\bf{Telescope}$: %s\n$\\bf{Camera}$: %s - $\\bf{Filter}$: %s - $\\bf{Integration}$: %s s\n$\\bf{Date time}$: %s'%(self.df_files.iloc[image_int]['object'],
                                                                                          self.df_files.iloc[image_int]['telescope'],
                                                                                          self.df_files.iloc[image_int]['camera'],
                                                                                          self.df_files.iloc[image_int]['filter'],
                                                                                          heads['INTEGT'],
                                                                                          self.df_files.iloc[image_int]['date_time'])
        ax.set_title(title_str)
        ax.minorticks_on()
        plt.tight_layout()
        plt.show()
    

    def read_data(self, image):
        """Method to view images."""
        image_str, image_int = self.return_index(image)
        print('Viewing ', image_str)

        # Extracting data from header
        with fits.open(os.path.join(self.dir_img, image_str)) as hdul:
            data = hdul[0].data.astype(np.float32)
            hdul.close()
        return data


In [150]:
iv = image_viewer('test_images', list_available=False, folder_list=['2025-11-02'])
# iv.df_files
# iv.header_info(1, interesting_keys='all')
# iv.view(4) #span_x=1000, span_y=1000)
iv.view(4, 
        centered = True,#('22h 40m 30.3s', '+3° 21′ 31″'),
        # zoom = ('0 1 d', '0:1 d'),
        stretch = 'sinh',
        percentile = (1,99),
        # vminmax = (2e4, None),
        cmap = 'gray',
        scalebar_arcsec= 5)

Current working directory: /Users/oscar/LB/grav_lens
Image directory defined: /Users/oscar/LB/grav_lens/test_images
Total number of images found:  6
Viewing  2025-11-02/TTT3_iKon936-1_2025-11-02-20-01-44-116500_EinsteinCross_SDSSr.fits


In [92]:
iv.header_info(4, interesting_keys='all')
# for k in range(5):
#     iv.header_info(k, interesting_keys=['CCW'])
#iv.view(1) #span_x=5000, span_y=5000)

Image: 2025-11-02/TTT3_iKon936-1_2025-11-02-20-01-44-116500_EinsteinCross_SDSSr.fits

   --- HEADER DATA ---
SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 2048                                                  
NAXIS2  =                 2048                                                  
IMAGETYP= 'SCIENCE '           / Type of image                                  
COMMENT ***************************                                             
COMMENT          TELESCOPE                                                      
COMMENT ***************************                                             
TELESCOP= 'TTT3    '           / Telescope name                                 
SITENAME= 'Teide Observatory (IAC)' / Telescope site name                       


In [74]:
iv.header_info(5, interesting_keys='all')

Image: 2025-11-02/TTT3_iKon936-1_2025-11-02-20-04-48-990118_EinsteinCross_SDSSr.fits

   --- HEADER DATA ---
SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 2048                                                  
NAXIS2  =                 2048                                                  
IMAGETYP= 'SCIENCE '           / Type of image                                  
COMMENT ***************************                                             
COMMENT          TELESCOPE                                                      
COMMENT ***************************                                             
TELESCOP= 'TTT3    '           / Telescope name                                 
SITENAME= 'Teide Observatory (IAC)' / Telescope site name                       


In [109]:
a = np.sqrt(6.3956041681758E-05**2 + 5.72564911657446E-07**2)
b = np.sqrt(6.01451624354911E-07**2 + 6.39515788549203E-05**2)

print(a*3600, b*3600)

0.23025097644173348 0.2302358654074819


In [116]:

im_i = 4
hdulist = fits.open(os.path.join(iv.dir_img,iv.return_index(im_i)[0]))
hdu = hdulist[0]
wcs = WCS(hdu.header)

fig, ax = plt.subplots(subplot_kw=dict(projection=wcs))
# Create an ImageNormalize object
snorm = SimpleNorm('sinh', vmin =1.8e4, vmax=3e4)#percent = 98)
axim = snorm.imshow(hdu.data, ax=ax, origin='lower')
fig.colorbar(axim)
#ax.imshow(hdu.data, origin='lower', cmap='viridis', norm = 'log')
ax.set_title('WCS plot of\n%s'%iv.return_index(im_i)[0])
ax.coords.grid(color='white', alpha=0.5, linestyle='solid')

# Add a scale bar
scalebar_angle = 5/3600*u.deg
add_scalebar(ax, scalebar_angle, label="5 arcsec", color="white")
ax.set(xlabel='RA', ylabel='Dec')
ax.scatter(340.1265440130891*u.deg, 3.357754897693203*u.deg, transform = ax.get_transform('icrs'), s=300)
ax.scatter(Angle('22 40 30.234 hours').to(u.deg), Angle('+03:21:30.63 degrees'), transform = ax.get_transform('icrs'), s=200, color = 'red')
plt.show()

qt.qpa.backingstore: Back buffer dpr of 2 doesn't match <NSViewBackingLayer: 0x8cb1f3250> contents scale of 1 - updating layer to match.
Traceback (most recent call last):
  File "/Users/oscar/LB/grav_lens/.venv/lib/python3.13/site-packages/matplotlib/backends/backend_qt.py", line 523, in _draw_idle
    self.draw()
    ~~~~~~~~~^^
  File "/Users/oscar/LB/grav_lens/.venv/lib/python3.13/site-packages/matplotlib/backends/backend_agg.py", line 382, in draw
    self.figure.draw(self.renderer)
    ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^
  File "/Users/oscar/LB/grav_lens/.venv/lib/python3.13/site-packages/matplotlib/artist.py", line 94, in draw_wrapper
    result = draw(artist, renderer, *args, **kwargs)
  File "/Users/oscar/LB/grav_lens/.venv/lib/python3.13/site-packages/matplotlib/artist.py", line 71, in draw_wrapper
    return draw(artist, renderer)
  File "/Users/oscar/LB/grav_lens/.venv/lib/python3.13/site-packages/matplotlib/figure.py", line 3257, in draw
    mimage._draw_list_compositing_image

In [88]:
Angle('22 40 30.234 hours').to(u.deg)

<Angle 340.125975 deg>

In [49]:
for im_i, filter in zip([3,4],['g','r']):
    hdulist = fits.open(os.path.join(iv.dir_img,iv.return_index(im_i)[0]))
    hdu = hdulist[0]
    globals()[filter] = hdu.data
    hdulist.close()
def normalize(data):
    data_min, data_max = np.percentile(data, (1, 99))
    data = np.clip(data, data_min, data_max)
    return (data - data_min) / (data_max - data_min)
g_n = normalize(g)
r_n = normalize(r)
i = np.zeros_like(g)
rgb_default = make_lupton_rgb(g_n, r_n, i)#, filename="ngc6976-default.jpeg")
fig, ax = plt.subplots()
ax.imshow(rgb_default, origin='lower')
fig.colorbar(ax)

/Users/oscar/LB/grav_lens/.venv/lib/python3.13/site-packages/astropy/visualization/lupton_rgb.py:645: RuntimeWarning: invalid value encountered in divide
  fInorm = np.where(Int <= 0, 0, np.true_divide(fI, Int))


AttributeError: 'Axes' object has no attribute 'cmap'

In [190]:
iv.header_info(1, interesting_keys='all')
#iv.view(2, span_x = 3000, span_y = 3000, center_xy = [7000, 5000]) #span=(400,400), print_info = True)

Image: TTT1_QHY411-1_2025-01-08-23-13-10-194024_SDSSJ0819+5356_SDSSr.fits

   --- HEADER DATA ---
SIMPLE  =  True   ---   conforms to FITS standard
BITPIX  =  -32   ---   array data type
NAXIS  =  2   ---   number of array dimensions
NAXIS1  =  14200   ---   
NAXIS2  =  10650   ---   
GPSDAT  =  2025-01-08T23:13:10.194024   ---   
GPSLAT  =  28.2987605   ---   
GPSLON  =  -16.50956333333333   ---   
SEQNUM  =  2   ---   
IMAGETYP  =  SCIENCE   ---   Type of image
TELESCOP  =  TTT1   ---   Telescope name
SITENAME  =  Teide Observatory (IAC)   ---   Telescope site name
SITECODE  =  Y65   ---   Telescope IAU Code
SITELAT  =  28.29871121   ---   [deg] Telescope latitude
SITELONG  =  -16.50956893   ---   [deg] Telescope longitude
SITEELEV  =  2359.12   ---   [m] Telescope elevation
DIAMETER  =  800.0   ---   [mm] Telescope diameter
FOCAL  =  6.85   ---   Telescope focal ratio
FOCALEN  =  5480.0   ---   [mm] Telescope focal length
MOUNT  =  AltAz   ---   Mount type
TRACK  =  1   ---   Sidere

# Work with historic dataframe imported from .pkl file

In [20]:
df = pd.read_pickle('df_files_all.pkl')
#df.set_index('date_time').groupby('object').resample('D').size().unstack(0)
df['day'] = df['date_time'].dt.date
df

,filename,path,telescope,camera,object,filter,size_MB,date_time,day
0,ATLAS-Teide-P_2023-05-01-22-01-07-273279_AL62....,/home/oscarsoler/work/red/2023-05-01/ATLAS-Tei...,err,err,err,,244.37664,2000-01-01 00:00:00.000000,2000-01-01
1,ATLAS-Teide-P_2023-05-02-04-51-46-695793_AM82....,/home/oscarsoler/work/red/2023-05-01/ATLAS-Tei...,err,err,err,,244.37088,2000-01-01 00:00:00.000000,2000-01-01
2,ATLAS-Teide-P_2023-05-04-00-05-04-730911_Focus...,/home/oscarsoler/work/red/2023-05-03/ATLAS-Tei...,err,err,err,,244.37664,2000-01-01 00:00:00.000000,2000-01-01
3,ATLAS-Teide-P_2023-05-04-00-23-31-426872_Focus...,/home/oscarsoler/work/red/2023-05-03/ATLAS-Tei...,err,err,err,,244.37088,2000-01-01 00:00:00.000000,2000-01-01
4,ATLAS-Teide-P_2023-05-04-00-32-10-828223_Focus...,/home/oscarsoler/work/red/2023-05-03/ATLAS-Tei...,err,err,err,,244.37088,2000-01-01 00:00:00.000000,2000-01-01
...,...,...,...,...,...,...,...,...,...
1357711,TTT3_iKon936-1_2025-10-31-06-43-58-638998_TOI-...,/home/oscarsoler/work/red/2025-10-30/TTT3_iKon...,TTT3,iKon936-1,TOI-2267,SDSSzs,16.79040,2025-10-31 06:43:58.638998,2025-10-31
1357712,TTT3_iKon936-1_2025-10-31-06-44-28-510311_TOI-...,/home/oscarsoler/work/red/2025-10-30/TTT3_iKon...,TTT3,iKon936-1,TOI-2267,SDSSzs,16.79040,2025-10-31 06:44:28.510311,2025-10-31
1357713,TTT3_iKon936-1_2025-10-31-06-44-58-381037_TOI-...,/home/oscarsoler/work/red/2025-10-30/TTT3_iKon...,TTT3,iKon936-1,TOI-2267,SDSSzs,16.79040,2025-10-31 06:44:58.381037,2025-10-31
1357714,TTT3_iKon936-1_2025-10-31-06-45-28-250995_TOI-...,/home/oscarsoler/work/red/2025-10-30/TTT3_iKon...,TTT3,iKon936-1,TOI-2267,SDSSzs,16.79040,2025-10-31 06:45:28.250995,2025-10-31


In [27]:
grav_lens = ['QSO0957+561', 'Q2237+030', 'MG1654+1346', 'SDSSJ1004+4112', 'LBQS1333+0113', 'SDSSJ0819+5356',
             'EinsteinCross']

# for g in grav_lens:
#     try: print(g, df.value_counts(['object']).loc[(g)])
#     except: print()


# Filter the DataFrame
df_filtered = df[df["object"].isin(grav_lens)].copy()

# Group by day and object, count observations
daily_counts = (
    df_filtered.groupby(["day", "object"])
    .size()
    .reset_index(name="count")
    )

# daily_counts.groupby(['filter']).size()


## Timeline overview of gravitational lens object observations

In [131]:

# Pivot for plotting
pivoted = daily_counts.pivot(index="day", columns="object", values="count").fillna(0)
pivot_for_plot = pivoted.where(pivoted != 0, np.nan)
# Remove days where all objects are NaN (no observations at all)
pivot_for_plot = pivot_for_plot.dropna(how="all")
# optional: sort index (dates) to ensure proper time order
pivot_for_plot = pivot_for_plot.sort_index()
# Plot
fig, ax = plt.subplots()

pivot_for_plot.plot(kind="line", marker="o", lw=0, alpha=0.5, figsize=(10, 5), ax=ax)


ax.set_title("Daily Observations by Object")
ax.minorticks_on()
ax.set_xlabel("Date")
ax.tick_params('x', rotation = 45)
ax.set_ylabel("Number of Observations")
ax.legend(title="Object")
ax.grid(alpha=0.5)
plt.tight_layout()
plt.show()

qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <NSViewBackingLayer: 0x11a229690> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <NSViewBackingLayer: 0x32ffbe780> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <NSViewBackingLayer: 0x11a229690> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <NSViewBackingLayer: 0x32ffbe780> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 2 doesn't match <NSViewBackingLayer: 0x11a229690> contents scale of 1 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <NSViewBackingLayer: 0x32ffbe780> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <NSViewBackingLayer: 0x32ffbe780> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1

In [94]:
plt.savefig('grav_lens_overview_object_date.png', dpi=200)

(0.12156862745098039, 0.4666666666666667, 0.7058823529411765)

## Overview of timeline observations separated by filter for each object

In [130]:
# Assiging each filter a color
filters_list = df_filtered.groupby('filter').size().keys().to_list()
n_filters = len(filters_list)

colors_dict = {filters_list[i]: plt.cm.tab10.colors[i] for i in range(n_filters)}
colors_list = [plt.cm.tab10.colors[i] for i in range(n_filters)]

# Create figure
# CORRECT to adjust to total number of filters
fig, ax = plt.subplots(4, 2)

for i, obj in enumerate(grav_lens):
    # Group by date, object, filter and count observations
    summary = (
        df_filtered[df_filtered['object']==obj].groupby(['day', 'object', 'filter'])
        .size()
        .reset_index(name='n_observations')
    )
    # Pivot the grouped data: rows are dates, columns are (object, filter) multiindex, values are counts
    pivoted = summary.pivot_table(
                                index='day',
                                columns=['object', 'filter'],
                                values='n_observations',
                                fill_value=0
)
# Plot as stacked bar chart
    #piv_plot = pivoted[pivoted['object']==obj].copy()
    iax = i//2
    jax = i%2
    # extract filters used for the object
    filters_object = summary.groupby('filter').size().keys().to_list()
    colors_object = [colors_dict.get(f, '#333333') for f in filters_object]
    pivoted.plot(
        kind='bar',
        stacked=True,
        figsize=(15, 7),
        ax=ax[iax, jax],
        legend = False,
        color = colors_object
    )
    ax[iax, jax].locator_params(axis='x', nbins=5)
    ax[iax, jax].tick_params('x', rotation = 0)
    ax[iax, jax].set_xlabel('')
    ax[iax, jax].set_title(obj)

    if iax == ax.shape[0]-1:
        ax[iax, jax].set_xlabel('Observation Date')
    if jax==0:
       ax[iax, jax].set_ylabel('Number of\nObservations')
    else:
        ax[iax, jax].set_ylabel('')

fig.suptitle('Daily Number of Observations per Object and Filter')
# Custom legend handles
ax[-1,-1].set_axis_off()
handles = [patches.Patch(color = colors_list[i], label =filters_list[i]) for i in range(n_filters)]
ax[-1,-1].legend(title='Filter', #bbox_to_anchor=(1.05, 1), loc='lower center',
           handles = handles, ncols = n_filters)
plt.tight_layout()
plt.show()

In [92]:
plt.savefig('grav_lens_overview_object_date_filter.png', dpi=200)

In [96]:
filters_list

['SDSSg', 'SDSSi', 'SDSSr', 'SDSSu', 'SDSSy', 'SDSSzs']

In [125]:
df_filtered.groupby(['object', 'filter', 'telescope']).size()

object          filter  telescope
EinsteinCross   SDSSg   TTT3           28
                SDSSr   TTT3           28
LBQS1333+0113   SDSSg   TTT1           65
                        TTT2           90
                SDSSi   TTT1           65
                        TTT2           90
                SDSSr   TTT1           65
                        TTT2           90
MG1654+1346     SDSSg   TTT1          158
                        TTT2          469
                        TTT3          322
                SDSSi   TTT1          159
                        TTT2          468
                        TTT3          322
                SDSSr   TTT1          158
                        TTT2          468
                        TTT3          322
Q2237+030       SDSSg   TTT3           77
                SDSSi   TTT3           77
                SDSSr   TTT3           77
                SDSSzs  TTT3           77
QSO0957+561     SDSSg   TST          1388
                        TTT1         1744


# Future work

In [ ]:
# ------- FUTURE WORK: IMAGE DATA HISTOGRAM ---------
# data = iv.read_data(1)
print(np.mean(data))
print(np.median(data))
print(np.std(data))
print(np.max(data),'-', np.min(data))
plt.hist(data.ravel(), bins=100)
plt.yscale('log')
plt.xscale('log')
plt.show()